In [26]:
import os

# datasets_path = os.path.join('..', 'datasets')
path = 'mongo.comment.csv'

In [27]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import trim, col

# from pyspark.ml.tuning import TrainValidateSplit, ParamGridBuilder
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id 

df = sqlContext.read.csv(path, header=False) # requires spark 2.0
df.show()
df.count()


+------------+--------------+------+
|         _c0|           _c1|   _c2|
+------------+--------------+------+
|    authorID|          name|rating|
|        joam|        添好運|    40|
|        Nash|        村子口|    35|
|      yu0226|    維記茶餐廳|    30|
|ipeen1199276|鼎泰豐(信義店)|    40|
|      nj1314|    香港茶水攤|    35|
|   eggface45|      天下三絕|    50|
|   wang12177|        添好運|    40|
|      yu0226|        村子口|    35|
|mandelicious|    永康牛肉麵|    35|
|ipeen1313278|    維記茶餐廳|    30|
| ipeen365625|    香港茶水攤|    50|
|    BSponpon|鼎泰豐(信義店)|    40|
|   hobandnob|      天下三絕|    40|
|mjstrong2002|        點點心|    35|
|ipeen1208434|      藍家割包|    45|
| leesunny989|  杭州小籠湯包|    25|
| ipeen572314|        添好運|    40|
|mandelicious|      宋廚菜館|    25|
|mandelicious|        鼎泰豐|    45|
+------------+--------------+------+
only showing top 20 rows



179455

In [28]:
from pyspark.sql.functions import trim

df = df.withColumn("_c0", trim(df._c0))
df = df.withColumn("_c1", trim(df._c1))
df = df.withColumn("_c2", trim(df._c2))
    
df.show()

+------------+--------------+------+
|         _c0|           _c1|   _c2|
+------------+--------------+------+
|    authorID|          name|rating|
|        joam|        添好運|    40|
|        Nash|        村子口|    35|
|      yu0226|    維記茶餐廳|    30|
|ipeen1199276|鼎泰豐(信義店)|    40|
|      nj1314|    香港茶水攤|    35|
|   eggface45|      天下三絕|    50|
|   wang12177|        添好運|    40|
|      yu0226|        村子口|    35|
|mandelicious|    永康牛肉麵|    35|
|ipeen1313278|    維記茶餐廳|    30|
| ipeen365625|    香港茶水攤|    50|
|    BSponpon|鼎泰豐(信義店)|    40|
|   hobandnob|      天下三絕|    40|
|mjstrong2002|        點點心|    35|
|ipeen1208434|      藍家割包|    45|
| leesunny989|  杭州小籠湯包|    25|
| ipeen572314|        添好運|    40|
|mandelicious|      宋廚菜館|    25|
|mandelicious|        鼎泰豐|    45|
+------------+--------------+------+
only showing top 20 rows



In [29]:
# read mysql_shop
df_shop = sqlContext.read.csv("mysql_shop.csv", header=True) # requires spark 2.0
df_shop = df_shop.withColumnRenamed("id", "shopid")
df_shop.show()
df_shop.count()

+------+------------------+
|shopid|              name|
+------+------------------+
|     1|            村子口|
|     2|        香港茶水攤|
|     3|        雙月食品社|
|     4|            添好運|
|     5|叁和院(忠孝旗艦店)|
|     6|    富霸王豬腳極品|
|     7|          龍都酒樓|
|     8|        永康牛肉麵|
|     9|        維記茶餐廳|
|    10|    鼎泰豐(信義店)|
|    11|            點點心|
|    12|          藍家割包|
|    13|      杭州小籠湯包|
|    14|    水舞饌(大直店)|
|    15|餡老滿(南港旗艦店)|
|    16|    牛店精燉牛肉麵|
|    17|        新港茶餐廳|
|    18|          天下三絕|
|    19|          宋廚菜館|
|    20|            鼎泰豐|
+------+------------------+
only showing top 20 rows



33253

In [30]:
df_shop = df_shop.withColumn("shopid", trim(df_shop.shopid))
df_shop = df_shop.withColumn("name", trim(df_shop.name))

df_shop.show()

+------+------------------+
|shopid|              name|
+------+------------------+
|     1|            村子口|
|     2|        香港茶水攤|
|     3|        雙月食品社|
|     4|            添好運|
|     5|叁和院(忠孝旗艦店)|
|     6|    富霸王豬腳極品|
|     7|          龍都酒樓|
|     8|        永康牛肉麵|
|     9|        維記茶餐廳|
|    10|    鼎泰豐(信義店)|
|    11|            點點心|
|    12|          藍家割包|
|    13|      杭州小籠湯包|
|    14|    水舞饌(大直店)|
|    15|餡老滿(南港旗艦店)|
|    16|    牛店精燉牛肉麵|
|    17|        新港茶餐廳|
|    18|          天下三絕|
|    19|          宋廚菜館|
|    20|            鼎泰豐|
+------+------------------+
only showing top 20 rows



In [31]:
# read mysql_commentator
df_comm = sqlContext.read.csv("mysql_commentator.csv", header=True) # requires spark 2.0
df_comm = df_comm.withColumnRenamed("id", "commid")
df_comm.show()
df_comm.count()
# df_comm.printSchema()

+------+---------------+
|commid|    commentator|
+------+---------------+
|     1|     吃比瘦有福|
|     2| Nash，神之領域|
|     3|           張小|
|     4|         快樂瑜|
|     5|         Choco♥|
|     6|     蛋寶趴趴go|
|     7|         王小玉|
|     8|       胖樺饌食|
|     9|大胖&小胖吃胖胖|
|    10|     SweetBunny|
|    11|      Lucy Chen|
|    12|   Angela's日嚐|
|    13|         饅頭弟|
|    14|     強強火車頭|
|    15|    sunny(晴兒)|
|    16|       summer47|
|    17|  Greta饗食趣樂|
|    18|     可愛小猴子|
|    19| 阿君的玩食天堂|
|    20|        寶寶750|
+------+---------------+
only showing top 20 rows



17071

In [32]:
df_comm = df_comm.withColumn("commid", trim(df_comm.commid))
df_comm = df_comm.withColumn("commentator", trim(df_comm.commentator))
df_comm.show()

+------+---------------+
|commid|    commentator|
+------+---------------+
|     1|     吃比瘦有福|
|     2| Nash，神之領域|
|     3|           張小|
|     4|         快樂瑜|
|     5|         Choco♥|
|     6|     蛋寶趴趴go|
|     7|         王小玉|
|     8|       胖樺饌食|
|     9|大胖&小胖吃胖胖|
|    10|     SweetBunny|
|    11|      Lucy Chen|
|    12|   Angela's日嚐|
|    13|         饅頭弟|
|    14|     強強火車頭|
|    15|    sunny(晴兒)|
|    16|       summer47|
|    17|  Greta饗食趣樂|
|    18|     可愛小猴子|
|    19| 阿君的玩食天堂|
|    20|        寶寶750|
+------+---------------+
only showing top 20 rows



In [33]:
df_cr = df.join(df_comm, df._c0 == df_comm.commentator, "inner")
df_cr.show()
df_cr.count()

+----------+--------------------------+---+------+-----------+
|       _c0|                       _c1|_c2|commid|commentator|
+----------+--------------------------+---+------+-----------+
|   kayfire|              穆記牛肉麵館| 15|   979|    kayfire|
|    simona|                    秦味館| 50|   687|     simona|
|  nightoro|                紅磡新飲茶| 25|   714|   nightoro|
|    yuning|            鼎泰豐(信義店)| 50|    72|     yuning|
|  JosefLee|                  銀翼餐廳| 45|  1459|   JosefLee|
|  luckbear|                    意麵王| 30|   130|   luckbear|
|    calamy|           六福客棧-金鳳廳| 35|   530|     calamy|
|    seagod|              劉山東牛肉麵| 35|   600|     seagod|
|  andy4990|        海霸王餐廳(中山店)| 35|   712|   andy4990|
|  nightoro|                開飯川食堂| 30|   714|   nightoro|
|   lun1203|          牛軋堂牛肉麵本家| 40|   734|    lun1203|
|  Naizigie|         南港 北大荒水餃店| 35|   739|   Naizigie|
|     Feena|                天雲扁食店| 35|  1122|      Feena|
|      1000|            香港波記茶餐廳| 35|  1127|       1000|
|sharonlia

8301

In [34]:
df_cr = df_cr.join(df_shop, df_cr._c1 == df_shop.name, "inner")
df_cr.show()
df_cr.count()

+----------+--------------------------+---+------+-----------+------+--------------------------+
|       _c0|                       _c1|_c2|commid|commentator|shopid|                      name|
+----------+--------------------------+---+------+-----------+------+--------------------------+
|   kayfire|              穆記牛肉麵館| 15|   979|    kayfire|    34|              穆記牛肉麵館|
|    simona|                    秦味館| 50|   687|     simona|    23|                    秦味館|
|  nightoro|                紅磡新飲茶| 25|   714|   nightoro|    82|                紅磡新飲茶|
|    yuning|            鼎泰豐(信義店)| 50|    72|     yuning|    10|            鼎泰豐(信義店)|
|  JosefLee|                  銀翼餐廳| 45|  1459|   JosefLee|   101|                  銀翼餐廳|
|  luckbear|                    意麵王| 30|   130|   luckbear|   106|                    意麵王|
|    calamy|           六福客棧-金鳳廳| 35|   530|     calamy|   100|           六福客棧-金鳳廳|
|    seagod|              劉山東牛肉麵| 35|   600|     seagod|    32|              劉山東牛肉麵|
|  andy4990| 

8285

In [35]:
# Convert cid, rid, _c2 columns to Long

import pyspark.sql.types

FEATURES_COL = ['commid', 'shopid', '_c2']

# print(df_als.columns)
for col in df_cr.columns:
    if col in FEATURES_COL:
        df_cr = df_cr.withColumn(col,df_cr[col].cast('long'))
df_cr.show()

+----------+--------------------------+---+------+-----------+------+--------------------------+
|       _c0|                       _c1|_c2|commid|commentator|shopid|                      name|
+----------+--------------------------+---+------+-----------+------+--------------------------+
|   kayfire|              穆記牛肉麵館| 15|   979|    kayfire|    34|              穆記牛肉麵館|
|    simona|                    秦味館| 50|   687|     simona|    23|                    秦味館|
|  nightoro|                紅磡新飲茶| 25|   714|   nightoro|    82|                紅磡新飲茶|
|    yuning|            鼎泰豐(信義店)| 50|    72|     yuning|    10|            鼎泰豐(信義店)|
|  JosefLee|                  銀翼餐廳| 45|  1459|   JosefLee|   101|                  銀翼餐廳|
|  luckbear|                    意麵王| 30|   130|   luckbear|   106|                    意麵王|
|    calamy|           六福客棧-金鳳廳| 35|   530|     calamy|   100|           六福客棧-金鳳廳|
|    seagod|              劉山東牛肉麵| 35|   600|     seagod|    32|              劉山東牛肉麵|
|  andy4990| 

In [36]:
df_als = df_cr.select('commid', 'shopid', '_c2')
df_als.show()
df_als.printSchema()

# df_als.filter("cid = 0").show()

+------+------+---+
|commid|shopid|_c2|
+------+------+---+
|   979|    34| 15|
|   687|    23| 50|
|   714|    82| 25|
|    72|    10| 50|
|  1459|   101| 45|
|   130|   106| 30|
|   530|   100| 35|
|   600|    32| 35|
|   712|    47| 35|
|   714|    48| 30|
|   734|    25| 40|
|   739|    52| 35|
|  1122|    69| 35|
|  1127|    66| 35|
|  1126|    64| 40|
|   130|    81| 35|
|   362|    78| 35|
|  1140|    69| 25|
|  1141|    66| 30|
|   530|   100| 40|
+------+------+---+
only showing top 20 rows

root
 |-- commid: long (nullable = true)
 |-- shopid: long (nullable = true)
 |-- _c2: long (nullable = true)



In [37]:
df_als = df_als.withColumnRenamed("commid", "user")
df_als = df_als.withColumnRenamed("shopid", "item")
df_als = df_als.withColumnRenamed("_c2", "rating")

In [38]:
# ratings = spark.createDataFrame(ratingsRDD)
(training, test) = df_als.randomSplit([0.8, 0.2])

training.show()
training.printSchema()
test.show()

+----+-----+------+
|user| item|rating|
+----+-----+------+
|  72|   10|    50|
|  72|  602|    50|
|  72| 6649|    45|
|  72| 6804|    50|
|  72|11359|    45|
|  72|13039|    50|
|  72|13083|    50|
| 127|   12|    25|
| 127|  145|    30|
| 127|  370|    15|
| 127|  437|    30|
| 127|  465|    30|
| 127|  711|    25|
| 127| 1141|    25|
| 127| 4266|    25|
| 127| 6554|    35|
| 127| 6599|    35|
| 127| 6802|    30|
| 127| 8839|    20|
| 127| 8844|    30|
+----+-----+------+
only showing top 20 rows

root
 |-- user: long (nullable = true)
 |-- item: long (nullable = true)
 |-- rating: long (nullable = true)

+----+-----+------+
|user| item|rating|
+----+-----+------+
| 127|  144|    25|
| 127|  154|    25|
| 127|  316|    30|
| 127| 6564|    25|
| 127| 6794|    25|
| 127|10862|    35|
| 127|11179|    40|
| 127|11183|    40|
| 127|11265|    35|
| 127|11438|    20|
| 127|13026|    45|
| 130|  106|    30|
| 130|  344|    35|
| 130|  426|    30|
| 130|  995|    35|
| 130| 1478|    35|
| 13

In [39]:
def tune_ALS(train_data, validation_data, maxIter, regParams, ranks):
    """
    grid search function to select the best model based on RMSE of
    validation data
    Parameters
    ----------
    train_data: spark DF with columns ['userId', 'movieId', 'rating']
    
    validation_data: spark DF with columns ['userId', 'movieId', 'rating']
    
    maxIter: int, max number of learning iterations
    
    regParams: list of float, one dimension of hyper-param tuning grid
    
    ranks: list of float, one dimension of hyper-param tuning grid
    
    Return
    ------
    The best fitted ALS model with lowest RMSE score on validation data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in regParams:
            # get ALS model
            als = ALS().setMaxIter(maxIter).setRank(rank).setRegParam(reg).setColdStartStrategy("drop")
            # train ALS model
            model = als.fit(train_data)
            # evaluate the model by computing the RMSE on the validation data
            predictions = model.transform(validation_data)
            evaluator = RegressionEvaluator(metricName="rmse",
                                            labelCol="rating",
                                            predictionCol="prediction")
            rmse = evaluator.evaluate(predictions)
            print('{} latent factors and regularization = {}: '
                  'validation RMSE is {}'.format(rank, reg, rmse))
            if rmse < min_error:
                min_error = rmse
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and '
          'regularization = {}'.format(best_rank, best_regularization))
    return best_model

regParams = [0.01, 0.05, 0.10]
ranks = [5, 10, 15, 20]
tune_ALS(training, test, 20, regParams, ranks)

5 latent factors and regularization = 0.01: validation RMSE is 31.925829609905996
5 latent factors and regularization = 0.05: validation RMSE is 23.216537293873476
5 latent factors and regularization = 0.1: validation RMSE is 19.691359733601253
10 latent factors and regularization = 0.01: validation RMSE is 28.423782941161836
10 latent factors and regularization = 0.05: validation RMSE is 19.863575127992437
10 latent factors and regularization = 0.1: validation RMSE is 18.249207802594352
15 latent factors and regularization = 0.01: validation RMSE is 27.535232571414426
15 latent factors and regularization = 0.05: validation RMSE is 20.15027034131691
15 latent factors and regularization = 0.1: validation RMSE is 18.8050550719163
20 latent factors and regularization = 0.01: validation RMSE is 27.095376624267992
20 latent factors and regularization = 0.05: validation RMSE is 19.01201643373598
20 latent factors and regularization = 0.1: validation RMSE is 17.803961810123326

The best model

ALS_e7102f006f13

In [40]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=20, regParam=0.2, userCol="user", itemCol="item", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 17.191200109985733


In [42]:
from pyspark.ml.recommendation import ALSModel
model_path = "/home/vovo/PycharmProjects/etl/als.model"
model.save(model_path)

In [43]:
model2 = ALSModel.load(model_path)

In [44]:
# Generate top 10 movie recommendations for each user
userRecs = model2.recommendForAllUsers(10)
userRecs.show()
# Generate top 10 user recommendations for each movie
movieRecs = model2.recommendForAllItems(10)
movieRecs.show()

+-----+--------------------+
| user|     recommendations|
+-----+--------------------+
| 1580|[[11178, 53.63652...|
| 2866|[[6691, 44.9588],...|
| 7982|[[11299, 51.91744...|
| 8389|[[24365, 41.18277...|
| 8592|[[30871, 40.19397...|
|  897|[[30310, 44.32952...|
| 1127|[[11255, 47.65014...|
| 1483|[[15366, 47.28492...|
| 7417|[[30843, 43.34922...|
| 8105|[[7241, 49.937042...|
| 9564|[[30856, 40.99270...|
|14514|[[31237, 35.05945...|
| 1618|[[11210, 51.53941...|
| 1650|[[25252, 46.13425...|
| 4391|[[13717, 51.7347]...|
|13009|[[25252, 55.19248...|
|14324|[[30871, 42.96401...|
| 7066|[[7484, 50.00861]...|
| 8743|[[11560, 37.27383...|
| 1977|[[169, 44.959526]...|
+-----+--------------------+
only showing top 20 rows

+-----+--------------------+
| item|     recommendations|
+-----+--------------------+
| 2142|[[9330, 38.6661],...|
|10817|[[8722, 39.948864...|
|11317|[[1389, 50.011253...|
|15447|[[14438, 53.68940...|
|15727|[[9330, 56.66809]...|
|15957|[[9330, 45.11045]...|
| 6620|[[14948, 4

In [45]:
# Generate top 5 item recommendations for a specified set of 3 users
users = df_als.select(als.getUserCol()).distinct().limit(1)
users.show()
userSubsetRecs = model2.recommendForUserSubset(users, 5)
userSubsetRecs.show(truncate=95)

+----+
|user|
+----+
|8440|
+----+

+----+-----------------------------------------------------------------------------------------------+
|user|                                                                                recommendations|
+----+-----------------------------------------------------------------------------------------------+
|8440|[[11560, 50.148895], [104, 49.805298], [15460, 48.047596], [30851, 46.856632], [30843, 46.02...|
+----+-----------------------------------------------------------------------------------------------+



In [46]:
# Generate top 10 user recommendations for a specified set of 3 items
items = df_als.select(als.getItemCol()).distinct().limit(3)
itemSubSetRecs = model2.recommendForItemSubset(items, 5)
itemSubSetRecs.show(truncate=100)

+----+------------------------------------------------------------------------------------------------+
|item|                                                                                 recommendations|
+----+------------------------------------------------------------------------------------------------+
|1697|[[2200, 35.358807], [3753, 34.948406], [11618, 31.129406], [1164, 29.790989], [2241, 25.592052]]|
|2453|   [[14438, 33.78289], [7804, 33.2455], [1564, 31.899134], [4793, 31.73844], [16573, 30.532589]]|
+----+------------------------------------------------------------------------------------------------+

